In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt


Setting environment for Precision WorkStation


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
stack = 'DEMO998'

In [104]:
# **(HUMAN)** Create meta data information for this brain

In [ ]:
%%writefile DEMO998_config.ini
[DEFAULT]
planar_resolution_um = 0.46
section_thickness_um = 20

In [4]:
# get_all_images.py [stack] [input_spec_json]

In [5]:
input_spec_json = '/home/yuncong/Brain/preprocess/DEMO998_input_spec.json'

In [6]:
! ./jp2_to_tiff.py DEMO998 {input_spec_json}

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
Found 4 images.

(None, u'raw') missing:
set([])
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [22]:
%%writefile /home/yuncong/Brain/preprocess/DEMO998_input_spec.ini
[DEFAULT]
version = None
resolution = raw
data_dir = /media/yuncong/BstemAtlasData/DEMO998
filepath_to_imageName_mapping = /media/yuncong/BstemAtlasData/DEMO998/(.*)?_lossless.jp2
imageName_to_filepath_mapping = /media/yuncong/BstemAtlasData/DEMO998/%%s_lossless.jp2

Overwriting /home/yuncong/Brain/preprocess/DEMO998_input_spec.ini


In [29]:
input_spec_ini = '/home/yuncong/Brain/preprocess/DEMO998_input_spec.ini'

In [37]:
def load_ini(fp):
    import configparser
    config = configparser.ConfigParser()
    config.read(fp)
    input_spec = dict(config.items('DEFAULT'))
    return input_spec

In [39]:
def parse_input_spec(input_spec_json):
    
    input_spec = load_json(input_spec_json)
    data_dirs = {}
    filepath_to_imageName_mapping = {}
    imageName_to_filepath_mapping = {}
    for spec_one_version in input_spec:
        vr = (spec_one_version['version'], spec_one_version['resolution'])
        data_dirs[vr] = spec_one_version['data_dir']
        filepath_to_imageName_mapping[vr] = spec_one_version['filepath_to_imageName_mapping']
        imageName_to_filepath_mapping[vr] = spec_one_version['imageName_to_filepath_mapping']
    
    # search for all images

    image_names_all_data_dirs_flattened = set([])
    image_names_all_data_dirs = {}
    for vr, data_dir in data_dirs.iteritems():
        if data_dir is None: continue
        image_names = set([])
        if vr in filepath_to_imageName_mapping:
            for fn in os.listdir(data_dir):
                g = re.search(filepath_to_imageName_mapping[vr], os.path.join(data_dir, fn))
                if g is not None:
                    img_name = g.groups()[0]
                    image_names.add(img_name)
                    image_names_all_data_dirs_flattened.add(img_name)
        image_names_all_data_dirs[vr] = image_names
        
    return {
    'data_dirs': data_dirs,
    'filepath_to_imageName_mapping': filepath_to_imageName_mapping,
    'imageName_to_filepath_mapping': imageName_to_filepath_mapping,
    'all_image_names': image_names_all_data_dirs_flattened,
    'image_names_all_data_dirs': image_names_all_data_dirs
    }

In [41]:
input_spec = parse_input_spec(input_spec_json)

In [42]:
print "Found %d images.\n" % len(input_spec['all_image_names'])

# Make sure the every image has all three channels.
for vr, img_names in input_spec['image_names_all_data_dirs'].iteritems():
    print "(version, resolution) tuple", vr, 'does not have the follwing images:'
    print input_spec['all_image_names'] - img_names

Found 4 images.

(version, resolution) tuple (None, u'raw') does not have the follwing images:
set([])


In [9]:
# jp2_to_tiff.py [stack] [input_spec_json]
# Expected output: tiff images of (none, raw) in the standard location.

In [43]:
create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=None, resol='raw'))

'/data/CSHL_data_processed/DEMO998/DEMO998_raw'

In [44]:
all_image_names = list(input_spec['all_image_names'])
imageName_to_filepath_mapping = input_spec['imageName_to_filepath_mapping']

In [45]:
# The KDU program automatically uses all cores, so we just set jobs_per_node = 1.
run_distributed('export LD_LIBRARY_PATH=%(kdu_dir)s:$LD_LIBRARY_PATH; %(kdu_bin)s -i \"%%(in_fp)s\" -o \"%%(out_fp)s\"' % \
                {'kdu_bin': KDU_EXPAND_BIN, 'kdu_dir': os.path.dirname(KDU_EXPAND_BIN)},
                kwargs_list={'in_fp': [imageName_to_filepath_mapping[(None, 'raw')] % img_name
                                       for img_name in all_image_names],
                             'out_fp': [DataManager.get_image_filepath_v2(stack=stack, prep_id=None,
                                        resol='raw', version=None, fn=img_name)
                                        for img_name in all_image_names]},
                argument_type='single',
                jobs_per_node=1,
                local_only=True)

rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# extract_channel.py [input_spec] [channel] [out_version]
# Expected output: Single-channel images of out_version in the standard location.

In [147]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
stack = DEMO998
prep_id = None
version = None
resol = raw

Overwriting input_spec.ini


In [30]:
! python ./extract_channel.py input_spec.ini 2 Ntb

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [19]:
inputs = all_image_names # image name list (str list)
channel = 2
out_version = 'Ntb'

# optional specifiers
stack = 'DEMO999'
prep_id = None
resol = 'raw'
in_version = None

In [20]:
create_if_not_exists(DataManager.get_image_dir_v2(stack=stack, prep_id=None, resol='raw', version=out_version))

'/data/CSHL_data_processed/DEMO999/DEMO999_raw_Ntb'

In [21]:
run_distributed('convert \"%%(in_fp)s\" -channel %(channel)s -separate \"%%(out_fp)s\"' % {'channel': 'RGB'[channel]},
                kwargs_list=[{'in_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, 
                                        resol=resol, version=in_version, fn=img_name),
                                       'out_fp': DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, 
                                        resol=resol, version=out_version, fn=img_name)}
                                       for img_name in all_image_names],
                argument_type='single',
                jobs_per_node=1,
                local_only=True)

rm -f ~/stderr_*; rm -f ~/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# rescale_image.py [input_spec] [[rescale_factor] | [width] [height]] [out_resol]
# raw_Ntb -> thumbnail_Ntb

In [145]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
stack = DEMO998
prep_id = None
version = Ntb
resol = raw

Overwriting temp.ini


In [146]:
! python rescale.py input_spec.ini thumbnail -f {1./32}

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_Ntb.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
Rescale: 1.75 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_process

In [23]:
inputs = all_image_names # image name list (str list)

stack = 'DEMO999'
prep_id = None
in_resol = 'raw'
version = 'Ntb'

rescale_factor = 1./32
out_resol = 'thumbnail'

In [24]:
for img_name in all_image_names:

    t = time.time()

    in_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, resol=in_resol, version=version, fn=img_name)
    out_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, resol=out_resol, version=version, fn=img_name)
    create_parent_dir_if_not_exists(out_fp)
    
    img = imread(in_fp)
    
    img_tb = img[::int(1./rescale_factor), ::int(1./rescale_factor)]
    imsave(out_fp, img_tb)

    # Alternative: ImageMagick introduces an artificial noisy stripe in the output image.
#     cmd = 'convert %(in_fp)s -scale 3.125%% %(out_fp)s' % {'in_fp': in_fp, 'out_fp': out_fp}
#     execute_command(cmd)
        
    sys.stderr.write("Rescale: %.2f seconds.\n" % (time.time() - t)) # ~20s / image

/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_Ntb.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
Rescale: 1.99 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_thumbnail_Ntb.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
Rescale: 2.07 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_Ntb.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
Rescale: 2.17 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/io/_io.py:132: UserWarning: /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD

In [58]:
# normalize_intensity.py [input_spec] [out_version]
# thumbnail_Ntb -> thumbnail_NtbNormalized

In [47]:
! python normalize_intensity.py -h

usage: normalize_intensity.py [-h] input_spec out_version

Linearly normalize intensity to between 0 and 255

positional arguments:
  input_spec   Input specification
  out_version  Output image version

optional arguments:
  -h, --help   show this help message and exit


In [42]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
stack = DEMO998
prep_id = None
version = Ntb
resol = thumbnail

Overwriting temp.ini


In [44]:
! python normalize_intensity.py input_spec.ini NtbNormalized

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
convert "/data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_Ntb.tif" -normalize -depth 8 "/data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_NtbNormalized/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_NtbNormalized.tif" 
return code: 0
Intensity normalize: 0.40 seconds.
convert "/data/CSHL_data_processed/DEMO9

In [25]:
inputs = all_image_names # image name list (str list)

stack = 'DEMO999'
prep_id = None
resol = 'thumbnail'
in_version = 'Ntb'

out_version = 'NtbNormalized'

In [26]:
for img_name in all_image_names:

    t = time.time()

    in_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, resol=resol, version=in_version, fn=img_name)
    out_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, resol=resol, version=out_version, fn=img_name)
    create_parent_dir_if_not_exists(out_fp)
        
    cmd = """convert "%(in_fp)s" -normalize -depth 8 "%(out_fp)s" """ % {'in_fp': in_fp, 'out_fp': out_fp}
    execute_command(cmd)
    
    sys.stderr.write("Intensity normalize: %.2f seconds.\n" % (time.time() - t))

convert "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_Ntb.tif" -normalize -depth 8 "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_NtbNormalized/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_NtbNormalized.tif" 
return code: 0
Intensity normalize: 0.15 seconds.
convert "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_thumbnail_Ntb.tif" -normalize -depth 8 "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_NtbNormalized/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_thumbnail_NtbNormalized.tif" 
return code: 0
Intensity normalize: 0.10 seconds.
convert "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_Ntb.tif" -normalize -depth 8 "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_NtbNormalized.tif" 
return co

In [61]:
# **(HUMAN)**: browse thumbnails to verify orientations are all correct

In [ ]:
# align_compose_warp.py [input_specifiers] [anchor_img_name] [elastix_output_dir] [custom_output_dir]
# thumbnail -> prep1_thumbnail

In [53]:
# (HUMAN) Obtain roughly correctly sorted list of image names.

In [ ]:
# Step 1: Compute pairwise transforms.
# align.py [input_spec] [elastix_output_dir] [param_fp]

In [81]:
! python align.py -h

usage: align.py [-h] input_spec elastix_output_dir param_fp

Align consecutive images. Possible bad alignment pairs are written into a separate file.

positional arguments:
  input_spec          txt. Each row is prev_img_name, prev_fp, curr_img_name,
                      curr_fp
  elastix_output_dir  output dir. Files for each pairwise transform are stored
                      in sub-folder <currImageName>_to_<prevImageName>.
  param_fp            elastix parameter file path

optional arguments:
  -h, --help          show this help message and exit


In [82]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = NtbNormalized
resol = thumbnail

Overwriting temp.ini


In [83]:
elastix_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output')

In [84]:
param_fp = '/home/yuncong/Brain/preprocess/parameters/Parameters_Rigid_MutualInfo_noNumberOfSpatialSamples_4000Iters.txt'

In [85]:
! python align.py input_spec.ini {elastix_output_dir} {param_fp}

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [97]:
input_image_names = all_image_names

stack = 'DEMO999'
resol = 'thumbnail'
version = 'NtbNormalized'
prep_id = None

elastix_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output')

param_fp = '/home/yuncong/Brain/preprocess/parameters/Parameters_Rigid_MutualInfo_noNumberOfSpatialSamples_4000Iters.txt'

In [93]:
input_spec = 'DEMO999_align_input_spec.txt'

In [90]:
filelist = [(imgName, DataManager.get_image_filepath_v2(stack=stack, fn=imgName, 
                                                        prep_id=prep_id,
                                                        resol=resol, version=version))
             for imgName in input_image_names]

In [88]:
# np.savetxt(input_spec, filelist, fmt='%s')

In [95]:
filelist = np.loadtxt(input_spec, dtype=str).tolist()

In [96]:
t = time.time()
print 'Align...'

run_distributed("%(script)s \"%(output_dir)s\" \'%%(kwargs_str)s\' -p %(param_fp)s -r" % \
                {'script': os.path.join(REPO_DIR, 'preprocess', 'align_sequential.py'),
                'output_dir': elastix_output_dir,
                 'param_fp': param_fp
                },
                kwargs_list=[{'prev_img_name': filelist[i-1][0],
                              'curr_img_name': filelist[i][0],
                              'prev_fp': filelist[i-1][1],
                              'curr_fp': filelist[i][1],
                             }
                            for i in range(1, len(filelist))],
                argument_type='list',
                jobs_per_node=8,
               local_only=True)

Align...


rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# Step 2: Compose pairwise transforms to get each image's transform to the anchor image.
# compose.py [elastix_output_dir] [custom_output_dir] [input_spec] [anchor_img_name] --out [toanchor_transforms_fp]

In [86]:
! python compose.py -h

usage: compose.py [-h] [--elastix_output_dir ELASTIX_OUTPUT_DIR]
                  [--custom_output_dir CUSTOM_OUTPUT_DIR]
                  [--input_spec INPUT_SPEC]
                  [--anchor_img_name ANCHOR_IMG_NAME] [--out OUT]

Generate a pkl file that stores a dict. Keys are image names and values are (3,3)-matrices.

optional arguments:
  -h, --help            show this help message and exit
  --elastix_output_dir ELASTIX_OUTPUT_DIR
  --custom_output_dir CUSTOM_OUTPUT_DIR
  --input_spec INPUT_SPEC
  --anchor_img_name ANCHOR_IMG_NAME
  --out OUT


In [ ]:
# (HUMAN) select anchor name

In [ ]:
%%writefile {DataManager.get_anchor_filename_filename(stack=stack)}
MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304

In [87]:
anchor_img_name = 'MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304'

In [88]:
elastix_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output')
custom_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_custom_output')

In [89]:
toanchor_transforms_fp = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.csv' % \
                         dict(stack=stack, anchor_fn=anchor_img_name))

In [90]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = NtbNormalized
resol = thumbnail

Overwriting input_spec.ini


In [91]:
! python compose.py --elastix_output_dir "{elastix_output_dir}" \
--custom_output_dir "{custom_output_dir}" \
--input_spec input_spec.ini  \
--anchor "{anchor_img_name}" \
--out "{toanchor_transforms_fp}"

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
Load elastix-computed transform: /data/CSHL_data_processed/DEMO998/DEMO998_elastix_output/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_to_MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301/TransformParameters.0.txt
Load elastix-computed transform: /data/CSHL_data_processed/DEMO998/DEMO998_elastix_output/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_to_MD662&661-F101-2017.06.06-22.05

In [521]:
elastix_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_elastix_output')
custom_output_dir = os.path.join(THUMBNAIL_DATA_DIR, stack, stack + '_custom_output')
input_spec = 'DEMO999_align_input_spec.txt'
anchor_img_name = 'MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304'

# [optional]
toanchor_transforms_fp = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.csv' % \
                         dict(stack=stack, anchor_fn=anchor_img_name))

In [522]:
image_name_list = [img_name for img_name, fp in np.loadtxt(input_spec, dtype=str).tolist()]

In [525]:
anchor_idx = image_name_list.index(anchor_img_name)

transformation_to_previous_sec = {}

for i in range(1, len(image_name_list)):

    transformation_to_previous_sec[i] = DataManager.load_consecutive_section_transform(moving_fn=image_name_list[i], fixed_fn=image_name_list[i-1], elastix_output_dir=elastix_output_dir, custom_output_dir=custom_output_dir)

transformation_to_anchor_sec = {}

for moving_idx in range(len(image_name_list)):

    if moving_idx == anchor_idx:
        # transformation_to_anchor_sec[moving_idx] = np.eye(3)
        transformation_to_anchor_sec[image_name_list[moving_idx]] = np.eye(3)

    elif moving_idx < anchor_idx:
        T_composed = np.eye(3)
        for i in range(anchor_idx, moving_idx, -1):
            T_composed = np.dot(np.linalg.inv(transformation_to_previous_sec[i]), T_composed)
        # transformation_to_anchor_sec[moving_idx] = T_composed
        transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed

    else:
        T_composed = np.eye(3)
        for i in range(anchor_idx+1, moving_idx+1):
            T_composed = np.dot(transformation_to_previous_sec[i], T_composed)
        # transformation_to_anchor_sec[moving_idx] = T_composed
        transformation_to_anchor_sec[image_name_list[moving_idx]] = T_composed

    print moving_idx, image_name_list[moving_idx], transformation_to_anchor_sec[image_name_list[moving_idx]]

#################################################

# with open(toanchor_transforms_fp, 'w') as f:
#     pickle.dump(transformation_to_anchor_sec, f)
    # Note that the index starts at 0, BUT the .._renamed folder index starts at 1.
                                                                                     

0 MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305 [[ 9.99380157e-01 -3.52037246e-02  6.43611476e+01]
 [ 3.52037246e-02  9.99380157e-01 -2.29074725e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
1 MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
2 MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302 [[  0.99884395  -0.04807047  30.73029782]
 [  0.04807047   0.99884395 -46.30303942]
 [  0.           0.           1.        ]]
3 MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301 [[ 9.99843164e-01 -1.77100741e-02  2.56495999e+01]
 [ 1.77100741e-02  9.99843164e-01 -5.02563325e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


Load elastix-computed transform: /data/CSHL_data_processed/DEMO999/DEMO999_elastix_output/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_to_MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305/TransformParameters.0.txt
Load elastix-computed transform: /data/CSHL_data_processed/DEMO999/DEMO999_elastix_output/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_to_MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304/TransformParameters.0.txt
Load elastix-computed transform: /data/CSHL_data_processed/DEMO999/DEMO999_elastix_output/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_to_MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302/TransformParameters.0.txt


In [526]:
dict_to_csv(transformation_to_anchor_sec, toanchor_transforms_fp)

In [ ]:
# (HUMAN) set planar_resolution for DEMO998 in metadata.py

In [171]:
# Step 3: thumbnail -> prep1_thumbnail

# Single image
# warp_crop.py [input_fp] [output_fp] [--warp transform_str] [--crop box_xywh_str] [--pad_color pad_color]

# Multiple images
# warp_crop.py [input_spec] [out_prep_id] [--warp transforms_pkl] [--crop cropbox_json] [--pad_color pad_color]

# transforms_pkl: pickle of a Python dict. Keys are image names, values are (3,3) matrices that represent the transformations to the anchor.

In [92]:
! python warp_crop.py -h

usage: warp_crop.py [-h] [--input_spec INPUT_SPEC] [--out_prep_id OUT_PREP_ID]
                    [--input_fp INPUT_FP] [--output_fp OUTPUT_FP]
                    [--warp WARP] [--inverse_warp INVERSE_WARP] [--crop CROP]
                    [--pad_color PAD_COLOR] [-r INIT_ROTATE]

Warp and crop images.

optional arguments:
  -h, --help            show this help message and exit
  --input_spec INPUT_SPEC
                        json
  --out_prep_id OUT_PREP_ID
                        if not specified, assume None
  --input_fp INPUT_FP   input filepath
  --output_fp OUTPUT_FP
                        output filepath
  --warp WARP           pkl for multiple images or csv_str for one image
  --inverse_warp INVERSE_WARP
                        pkl for multiple images or csv_str for one image
  --crop CROP           json or box_xywh_str
  --pad_color PAD_COLOR
                        background color (black or white)
  -r INIT_ROTATE, --init_rotate INIT_ROTATE
                        escap

In [93]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = NtbNormalized
resol = thumbnail

Overwriting input_spec.ini


In [102]:
! python warp_crop.py --input_spec input_spec.ini \
 --warp "/data/CSHL_data_processed/DEMO999/DEMO999_transformsTo_MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304.csv" \
 --out_prep_id alignedPadded

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [270]:
def cropbox_to_list(cropbox):
    return np.array([cropbox['rostral_limit'], cropbox['dorsal_limit'], cropbox['caudal_limit'] -  cropbox['rostral_limit'] + 1, cropbox['ventral_limit'] -  cropbox['dorsal_limit'] + 1])

def list_to_str(s):
    return ','.join(map(str, s))

def cropbox_to_str(cropbox):
    return list_to_str(cropbox_to_list(cropbox))

def transform_to_str(transform):
    return ','.join(map(str, transform.flatten()))

def str_to_transform(transform_str):
    return np.reshape(map(np.float, transform_str.split(',')), (3,3))

def str_to_cropbox(cropbox_str):
    return map(int, map(eval, cropbox_str.split(',')))

def rescale_transform(transform, factor):

    T = transform.copy()
    T[:2,2] = transform[:2, 2] * factor
    return T

In [527]:
out_prep_id = 'alignedBrainstemCrop'
init_rotate = ''
pad_color = 'auto' # useful for alternatively stained stacks where bg varies depending on stain on each section

transforms_pkl = '/data/CSHL_data_processed/DEMO999/DEMO999_transformsTo_MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304.pkl'
warp = transforms_pkl

cropbox_json = '/data/CSHL_data_processed/DEMO999/DEMO999_alignedTo_MD662&661-F116-2017.06.07-04.39.41_MD661_1_0346_prep2_cropbox.json'
crop = cropbox_json

In [272]:
# input_spec = 'DEMO999_align_input_spec.txt'
input_spec = 'DEMO999_align_input_spec_v2.ini'

In [273]:
import configparser
config = configparser.ConfigParser()
config.read(input_spec)
input_spec = dict(config.items('DEFAULT'))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  This is separate from the ipykernel package so we can avoid doing imports until


In [274]:
image_name_list = input_spec['image_name_list'].split('\n')
stack = input_spec['stack']
prep_id = input_spec['prep_id']
version = input_spec['version']
resol = input_spec['resol']

# image_name_list = [img_name for img_name, fp in  np.loadtxt(input_spec, dtype=str).tolist()]
# stack = 'DEMO999'
# prep_id = None
# version = 'NtbNormalized'
# resol = 'thumbnail'

In [275]:
assert warp.endswith('.csv')
transforms_to_anchor = load_pickle(warp)

transforms_resol = 'thumbnail'

transforms_scale_factor = convert_resolution_string_to_um(stack=stack, resolution=transforms_resol) / convert_resolution_string_to_um(stack=stack, resolution=resol)
cropbox_scale_factor = convert_resolution_string_to_um(stack=stack, resolution=cropbox_resol) / convert_resolution_string_to_um(stack=stack, resolution=resol)

run_distributed('%(script)s --input_fp \"%%(input_fp)s\" --output_fp \"%%(output_fp)s\" --warp %%(transform_str)s --crop %%(box_xywh_str)s --pad_color %%(pad_color)s' % \
            {'script': os.path.join(REPO_DIR, 'preprocess', 'warp_crop.py'),
            },
            kwargs_list=[{'transform_str': transform_to_str(rescale_transform(transforms_to_anchor[img_name], factor=transforms_scale_factor)),
                      'box_xywh_str': list_to_str(cropbox_to_list(cropbox) * cropbox_scale_factor),
                    'input_fp': DataManager.get_image_filepath_v2(stack=stack, fn=img_name, prep_id=None, version=version, resol=resol),
                      'output_fp': DataManager.get_image_filepath_v2(stack=stack, fn=img_name, prep_id=out_prep_id, version=version, resol=resol),
                        'pad_color': ('black' if img_name.split('-')[1][0] == 'F' else 'white') if pad_color == 'auto' else pad_color}
                            for img_name in image_name_list],
            argument_type='single',
           jobs_per_node=8,
        local_only=True)

rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [276]:
image_name = 'MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302'

input_fp = DataManager.get_image_filepath_v2(stack=stack, fn=image_name, prep_id=prep_id, version=version, resol=resol)

output_fp = DataManager.get_image_filepath_v2(stack=stack, fn=image_name, prep_id=out_prep_id, version=version, resol=resol)
create_parent_dir_if_not_exists(output_fp)

transforms_to_anchor = load_pickle(transforms_pkl)
transform_str = transform_to_str(transforms_to_anchor[image_name])

cropbox_str = cropbox_to_str(load_json(cropbox_json))

warp = transform_str
crop = cropbox_str

In [278]:
if init_rotate == '':
    init_rotate = ''
else:
    init_rotate = orientation_argparse_str_to_imagemagick_str[init_rotate]

T = np.linalg.inv(str_to_transform(warp))
x, y, w, h = str_to_cropbox(crop)

execute_command("convert \"%(input_fp)s\" %(init_rotate)s +repage -virtual-pixel background -background %(bg_color)s +distort AffineProjection '%(sx)f,%(rx)f,%(ry)f,%(sy)f,%(tx)f,%(ty)f' -crop %(w)sx%(h)s%(x)s%(y)s\! -flatten -compress lzw \"%(output_fp)s\"" % \
            {'init_rotate':init_rotate,
                'sx':T[0,0],
 'sy':T[1,1],
 'rx':T[1,0],
'ry':T[0,1],
 'tx':T[0,2],
 'ty':T[1,2],
 'input_fp': input_fp,
 'output_fp': output_fp,
 'x': '+' + str(x) if int(x) >= 0 else str(x),
 'y': '+' + str(y) if int(y) >= 0 else str(y),
 'w': str(w),
'h': str(h),
 'bg_color': pad_color
}
)


convert "/data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_NtbNormalized.tif"  +repage -virtual-pixel background -background auto +distort AffineProjection '0.998844,-0.048070,0.048070,0.998844,-28.468963,47.726730' -crop 777x492+468+129\! -flatten -compress lzw "/data/CSHL_data_processed/DEMO999/DEMO999_prep2_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep2_thumbnail_NtbNormalized.tif"
return code: 0


In [ ]:
# **(HUMAN)** Inspect aligned images using preprocessGUI, correct pairwise transforms and check each image's order in stack. `preprocess_gui.py`

In [106]:
# (HUMAN) create DEMO998_sorted_filenames.txt

In [110]:
%%writefile {DataManager.get_sorted_filenames_filename(stack=stack)}
MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301 1
MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302 2
MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304 3
MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305 4

Overwriting /data/CSHL_data_processed/DEMO998/DEMO998_sorted_filenames.txt


In [ ]:
# (HUMAN) draw initial snake contours for masking

In [128]:
! python masking.py -h

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
usage: masking.py [-h] [--min_size MIN_SIZE]
                  [--default_channel DEFAULT_CHANNEL] [--shrink SHRINK]
                  input_spec init_snake_contours_fp

Generate masks for aligned thumbnail images

positional arguments:
  input_spec            stack name
  init_snake_contours_fp
                        initial snake contour file path

optional arguments:
  -h, --help 

In [129]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = alignedPadded
version = NtbNormalized
resol = thumbnail

Overwriting input_spec.ini


In [130]:
! python masking.py input_spec.ini {DataManager.get_initial_snake_contours_filepath(stack=stack)}

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_prep1_thumbnail_NtbNormalized.tif
Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep1_thumbnail_NtbNormalized.tif
Trying to load /data/CSHL_data_processed/DEMO998/

Final masks added.
aws s3 cp "/data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_autoSubmasks/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep1_thumbnail_autoSubmask_0.png" "s3://mousebrainatlas-data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_autoSubmasks/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep1_thumbnail_autoSubmask_0.png"
upload: ../../../../data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_autoSubmasks/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep1_thumbnail_autoSubmask_0.png to s3://mousebrainatlas-data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_autoSubmasks/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_prep1_thumbnail_autoSubmask_0.png
return code: 0
aws s3 cp "/data/CSHL_data_processed/DEMO998/DEMO998_prep1_thumbnail_autoSubmasks/MD66

In [ ]:
#(HUMAN) Return to masking GUI to inspect and correct the automatically generated masks.

In [ ]:
# prep1_thumbnail_mask -> thumbnail_mask

In [131]:
! python warp_crop.py --help

usage: warp_crop.py [-h] [--input_spec INPUT_SPEC] [--out_prep_id OUT_PREP_ID]
                    [--input_fp INPUT_FP] [--output_fp OUTPUT_FP]
                    [--warp WARP] [--inverse_warp INVERSE_WARP] [--crop CROP]
                    [--pad_color PAD_COLOR] [-r INIT_ROTATE]

Warp and crop images.

optional arguments:
  -h, --help            show this help message and exit
  --input_spec INPUT_SPEC
                        json
  --out_prep_id OUT_PREP_ID
                        if not specified, assume None
  --input_fp INPUT_FP   input filepath
  --output_fp OUTPUT_FP
                        output filepath
  --warp WARP           pkl for multiple images or csv_str for one image
  --inverse_warp INVERSE_WARP
                        pkl for multiple images or csv_str for one image
  --crop CROP           json or box_xywh_str
  --pad_color PAD_COLOR
                        background color (black or white)
  -r INIT_ROTATE, --init_rotate INIT_ROTATE
                        escap

In [143]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = alignedPadded
version = mask
resol = thumbnail

Overwriting input_spec.ini


In [133]:
d = {img_name: \
 (0,0) + DataManager.load_image_v2(stack=stack, prep_id=None, resol='thumbnail', version='NtbNormalized', fn=img_name).shape[::-1]
 for img_name in 
    [
        'MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301',
        'MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302',
        'MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304',
        'MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305',
    ]}

df = pd.DataFrame.from_dict({k: np.array(v).flatten() for k, v in d.iteritems()}, orient='index')
df.to_csv('/data/CSHL_data_processed/DEMO998/DEMO998_original_image_crop.csv', header=False)

Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_thumbnail_NtbNormalized.tif
Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_NtbNormalized/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_NtbNormalized.tif
Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_NtbNormalized/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_thumbnail_NtbNormalized.tif
Trying to load /data/CSHL_data_processed/DEMO998/DEMO998_thumbnail_NtbNormalized/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_NtbNormalized.tif


In [144]:
! python warp_crop.py --input_spec input_spec.ini \
 --inverse_warp "{toanchor_transforms_fp}" \
 --crop "/data/CSHL_data_processed/DEMO998/DEMO998_original_image_crop.csv" \
 --out_prep_id None

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# Brighfield correction
# normalize_intensity_adaptive.py [input_spec] [out_version]
# raw_Ntb -> raw_NtbNormalizedAdaptiveInvertedGamma

In [183]:
! python normalize_intensity_adaptive.py --help

usage: normalize_intensity_adaptive.py [-h] input_spec out_version

Linearly normalize intensity to between 0 and 255

positional arguments:
  input_spec   Input specification
  out_version  Output image version

optional arguments:
  -h, --help   show this help message and exit


In [184]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = Ntb
resol = raw

Overwriting input_spec.ini


In [188]:
! python normalize_intensity_adaptive.py input_spec.ini NtbNormalizedAdaptiveInvertedGamma

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will b

In [305]:
out_version = 'NtbNormalizedAdaptiveInvertedGamma'

image_name_list = all_image_names
stack = 'DEMO999'
prep_id = None
version = 'Ntb'
resol = 'raw'

In [301]:
from scipy.ndimage.interpolation import map_coordinates
from skimage.exposure import rescale_intensity, adjust_gamma
from skimage.transform import rotate

In [290]:
# for section in set(metadata_cache['valid_sections_all'][stack]) - set(metadata_cache['valid_sections'][stack]):
for image_name in all_image_names:
    
    print "Image", image_name
    
    t = time.time()
    
    img = DataManager.load_image_v2(stack=stack, prep_id=None, fn=image_name, version='Ntb', resol='thumbnail')

    sys.stderr.write('Load image: %.2f seconds.\n' % (time.time() - t))

#     t = time.time()
    tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=None, fn=image_name)
# #     raw_mask = rescale_by_resampling(tb_mask, new_shape=(img.shape[1], img.shape[0]))
#     raw_mask = resize(tb_mask, img.shape) > .5
    
#     save_data(raw_mask, 
#           DataManager.get_image_filepath_v2(stack=stack, prep_id=None, section=section, version='mask', resol='raw', ext='bp'), 
#           upload_s3=False)
    
#     sys.stderr.write('Rescale mask: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    
    mean_std_all_regions = []
    cx_cy_all_regions = []
    region_size = 5000/32
    region_spacing = 3000/32
#     for cx in range(region_size/2, img.shape[1]-region_size/2+1, region_spacing):
#         for cy in range(region_size/2, img.shape[0]-region_size/2+1, region_spacing):
    for cx in range(0, img.shape[1], region_spacing):
        for cy in range(0, img.shape[0], region_spacing):
            region = img[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                         max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
#             region_mask = raw_mask[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
#                                    max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            region_mask = tb_mask[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                                   max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            if np.count_nonzero(region_mask) == 0:
                continue
            mean_std_all_regions.append((region[region_mask].mean(), region[region_mask].std()))
            cx_cy_all_regions.append((cx, cy))
            
    sys.stderr.write('Compute mean/std for sample regions: %.2f seconds.\n' % (time.time() - t))
    
    t = time.time()
    mean_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,0], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4, 
                                 interpolation_order=2)

    sys.stderr.write('Interpolate mean map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    mean_map = rescale_by_resampling(mean_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up mean map: %.2f seconds.\n' % (time.time() - t)) #30s

    t = time.time()
    std_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,1], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4,
                               interpolation_order=2)
    sys.stderr.write('Interpolate std map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    std_map = rescale_by_resampling(std_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up std map: %.2f seconds.\n' % (time.time() - t)) #30s
    
#     # Save mean/std results.
    
#     fp = DataManager.get_intensity_normalization_result_filepath(what='region_centers', stack=stack, section=section)
#     create_parent_dir_if_not_exists(fp)    
#     np.savetxt(fp, cx_cy_all_regions)
    
#     fp = DataManager.get_intensity_normalization_result_filepath(what='mean_std_all_regions', stack=stack, section=section)
#     create_parent_dir_if_not_exists(fp)
#     np.savetxt(fp, mean_std_all_regions)
    
#     fp = DataManager.get_intensity_normalization_result_filepath(what='mean_map', stack=stack, section=section)
#     create_parent_dir_if_not_exists(fp)
#     bp.pack_ndarray_file(mean_map.astype(np.float16), fp)
    
#     fp = DataManager.get_intensity_normalization_result_filepath(what='std_map', stack=stack, section=section)
#     create_parent_dir_if_not_exists(fp)
#     bp.pack_ndarray_file(std_map.astype(np.float16), fp)

    # Export normalized image.
    
    t = time.time()
#     raw_mask = raw_mask & (std_map > 0)
#     img_normalized = np.zeros(img.shape, np.float32)
#     img_normalized[raw_mask] = (img[raw_mask] - mean_map[raw_mask]) / std_map[raw_mask]
    tb_mask = tb_mask & (std_map > 0)
    img_normalized = np.zeros(img.shape, np.float32)
    img_normalized[tb_mask] = (img[tb_mask] - mean_map[tb_mask]) / std_map[tb_mask]
    sys.stderr.write('Normalize: %.2f seconds.\n' % (time.time() - t)) #30s

#     t = time.time()
#     # FIX THIS! THIS only save uint16, not float16. Need to save as bp instead.
# #     img_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, version='NtbNormalizedFloat', resol='down8', section=section, )
# #     create_parent_dir_if_not_exists(img_fp)
# #     imsave(img_fp, img_normalized[::8, ::8].astype(np.float16))
#     save_data(img_normalized.astype(np.float16), 
#               DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, section=section),
#              upload_s3=False)
#     sys.stderr.write('Save float version: %.2f seconds.\n' % (time.time() - t)) #30s
        
# #     t = time.time()
# #     img_normalized_uint8 = rescale_intensity_v2(img_normalized, -1, 6)
# #     sys.stderr.write('Rescale to uint8: %.2f seconds.\n' % (time.time() - t)) #30s
    
# #     t = time.time()
# #     img_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, version='NtbNormalized', resol='raw', section=section)
# #     create_parent_dir_if_not_exists(img_fp)
# #     imsave(img_fp, img_normalized_uint8)
# #     sys.stderr.write('Save uint8 version: %.2f seconds.\n' % (time.time() - t)) #30s
    
#     # Export histogram.
    
#     plt.hist(img_normalized[raw_mask].flatten(), bins=100, log=True);
#     fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram_png', stack=stack, section=section)
#     create_parent_dir_if_not_exists(fp)
#     plt.savefig(fp)
#     plt.close();
    
# #     hist_fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram', stack=stack, section=section)
# #     create_parent_dir_if_not_exists(hist_fp)
    
# #     hist, bin_edges = np.histogram(img_normalized[valid_mask].flatten(), bins=np.arange(0,201,5));

# #     plt.bar(bin_edges[:-1], np.log(hist));
# #     plt.xticks(np.arange(0, 200, 20), np.arange(0, 200, 20));
# #     plt.xlabel('Normalized pixel value (float)');
# #     plt.title(metadata_cache['sections_to_filenames'][stack][section])

# #     plt.savefig(hist_fp)
# #     plt.close();

Image MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
Image MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
Image MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302


Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_thumbnail_Ntb.tif
Load image: 0.00 seconds.
Compute mean/std for sample regions: 0.03 seconds.
Interpolate mean map: 0.01 seconds.
Scale up mean map: 0.02 seconds.
Interpolate std map: 0.01 seconds.
Scale up std map: 0.02 seconds.
Normalize: 0.01 seconds.
Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_thumbnail_Ntb.tif
Load image: 0.00 seconds.
Compute mean/std for sample regions: 0.02 seconds.
Interpolate mean map: 0.01 seconds.
Scale up mean map: 0.02 seconds.
Interpolate std map: 0.01 seconds.
Scale up std map: 0.02 seconds.
Normalize: 0.01 seconds.
Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302_thumbnail_Ntb.tif
Load image: 0.00 seconds.
Compute mean/std for sample regions: 0.02 seconds.
Interpolate mean map: 0.01 seconds.


Image MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301


Scale up mean map: 0.01 seconds.
Interpolate std map: 0.01 seconds.
Scale up std map: 0.01 seconds.
Normalize: 0.01 seconds.
Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_thumbnail_Ntb/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_thumbnail_Ntb.tif
Load image: 0.00 seconds.
Compute mean/std for sample regions: 0.02 seconds.
Interpolate mean map: 0.01 seconds.
Scale up mean map: 0.01 seconds.
Interpolate std map: 0.01 seconds.
Scale up std map: 0.01 seconds.
Normalize: 0.01 seconds.


In [296]:
std_map.shape

(694, 1372)

In [299]:
# for section in set(metadata_cache['valid_sections_all'][stack]) - set(metadata_cache['valid_sections'][stack]):
# for section in metadata_cache['valid_sections'][stack]:

for image_name in all_image_names:

#     print "Section", section
    
    t = time.time()
    
    img = DataManager.load_image_v2(stack=stack, prep_id=prep_id, fn=image_name, version=version, resol=resol)

    sys.stderr.write('Load image: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    tb_mask = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=None, fn=image_name)
#     raw_mask = rescale_by_resampling(tb_mask, new_shape=(img.shape[1], img.shape[0]))
    raw_mask = resize(tb_mask, img.shape) > .5
    
    save_data(raw_mask, 
          DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, fn=image_name, version='mask', resol=resol, ext='bp'), 
          upload_s3=False)
    
    sys.stderr.write('Rescale mask: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    
    mean_std_all_regions = []
    cx_cy_all_regions = []
    region_size = 5000
    region_spacing = 3000
#     for cx in range(region_size/2, img.shape[1]-region_size/2+1, region_spacing):
#         for cy in range(region_size/2, img.shape[0]-region_size/2+1, region_spacing):
    for cx in range(0, img.shape[1], region_spacing):
        for cy in range(0, img.shape[0], region_spacing):
            region = img[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                         max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            region_mask = raw_mask[max(cy-region_size/2, 0):min(cy+region_size/2+1, img.shape[0]-1), 
                                   max(cx-region_size/2, 0):min(cx+region_size/2+1, img.shape[1]-1)]
            if np.count_nonzero(region_mask) == 0:
                continue
            mean_std_all_regions.append((region[region_mask].mean(), region[region_mask].std()))
            cx_cy_all_regions.append((cx, cy))
            
    sys.stderr.write('Compute mean/std for sample regions: %.2f seconds.\n' % (time.time() - t))
    
    t = time.time()
    mean_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,0], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4, 
                                 interpolation_order=2)

    sys.stderr.write('Interpolate mean map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    mean_map = rescale_by_resampling(mean_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up mean map: %.2f seconds.\n' % (time.time() - t)) #30s

    t = time.time()
    std_map = resample_scoremap(sparse_scores=np.array(mean_std_all_regions)[:,1], 
                             sample_locations=cx_cy_all_regions,
                             gridspec=(region_size, region_spacing, img.shape[1], img.shape[0], (0,0)),
                            downscale=4,
                               interpolation_order=2)
    sys.stderr.write('Interpolate std map: %.2f seconds.\n' % (time.time() - t)) #10s

    t = time.time()
    std_map = rescale_by_resampling(std_map, new_shape=(img.shape[1], img.shape[0]))
    sys.stderr.write('Scale up std map: %.2f seconds.\n' % (time.time() - t)) #30s
    
    # Save mean/std results.
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='region_centers', stack=stack, fn=image_name)
    create_parent_dir_if_not_exists(fp)    
    np.savetxt(fp, cx_cy_all_regions)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='mean_std_all_regions', stack=stack, fn=image_name)
    create_parent_dir_if_not_exists(fp)
    np.savetxt(fp, mean_std_all_regions)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='mean_map', stack=stack, fn=image_name)
    create_parent_dir_if_not_exists(fp)
    bp.pack_ndarray_file(mean_map.astype(np.float16), fp)
    
    fp = DataManager.get_intensity_normalization_result_filepath(what='std_map', stack=stack, fn=image_name)
    create_parent_dir_if_not_exists(fp)
    bp.pack_ndarray_file(std_map.astype(np.float16), fp)

    # Export normalized image.
    
    t = time.time()
    raw_mask = raw_mask & (std_map > 0)
    img_normalized = np.zeros(img.shape, np.float32)
    img_normalized[raw_mask] = (img[raw_mask] - mean_map[raw_mask]) / std_map[raw_mask]
    sys.stderr.write('Normalize: %.2f seconds.\n' % (time.time() - t)) #30s

    t = time.time()
    # FIX THIS! THIS only save uint16, not float16. Need to save as bp instead.
#     img_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, version='NtbNormalizedFloat', resol='down8', section=section, )
#     create_parent_dir_if_not_exists(img_fp)
#     imsave(img_fp, img_normalized[::8, ::8].astype(np.float16))
    save_data(img_normalized.astype(np.float16), 
              DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, fn=image_name),
             upload_s3=False)
    sys.stderr.write('Save float version: %.2f seconds.\n' % (time.time() - t)) #30s
        
#     t = time.time()
#     img_normalized_uint8 = rescale_intensity_v2(img_normalized, -1, 6)
#     sys.stderr.write('Rescale to uint8: %.2f seconds.\n' % (time.time() - t)) #30s
    
#     t = time.time()
#     img_fp = DataManager.get_image_filepath_v2(stack=stack, prep_id=None, version='NtbNormalized', resol='raw', section=section)
#     create_parent_dir_if_not_exists(img_fp)
#     imsave(img_fp, img_normalized_uint8)
#     sys.stderr.write('Save uint8 version: %.2f seconds.\n' % (time.time() - t)) #30s
    
    # Export histogram.
    
    plt.hist(img_normalized[raw_mask].flatten(), bins=100, log=True);
    fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram_png', stack=stack, fn=image_name)
    create_parent_dir_if_not_exists(fp)
    plt.savefig(fp)
    plt.close();
    
#     hist_fp = DataManager.get_intensity_normalization_result_filepath(what='float_histogram', stack=stack, section=section)
#     create_parent_dir_if_not_exists(hist_fp)
    
#     hist, bin_edges = np.histogram(img_normalized[valid_mask].flatten(), bins=np.arange(0,201,5));

#     plt.bar(bin_edges[:-1], np.log(hist));
#     plt.xticks(np.arange(0, 200, 20), np.arange(0, 200, 20));
#     plt.xlabel('Normalized pixel value (float)');
#     plt.title(metadata_cache['sections_to_filenames'][stack][section])

#     plt.savefig(hist_fp)
#     plt.close();

Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_raw_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305_raw_Ntb.tif
Load image: 2.36 seconds.
/usr/local/lib/python2.7/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Rescale mask: 29.54 seconds.
Compute mean/std for sample regions: 11.36 seconds.
Interpolate mean map: 7.11 seconds.
Scale up mean map: 18.18 seconds.
Interpolate std map: 7.18 seconds.
Scale up std map: 17.64 seconds.
Normalize: 10.94 seconds.
Save float version: 9.09 seconds.
Trying to load /data/CSHL_data_processed/DEMO999/DEMO999_raw_Ntb/MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_raw_Ntb.tif
Load image: 2.77 seconds.
Rescale mask: 31.94 seconds.
Compute mean/std for sample regions: 11.84 seconds.
Interpolate mean map: 8.51 seconds.
Scale up mean map: 18.46 seconds.
Interpolate std map: 8.59 seconds.
S

In [302]:
gamma_map = img_as_ubyte(adjust_gamma(np.arange(0, 256, 1) / 255., 8.))

/usr/local/lib/python2.7/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [303]:
low = -2.
high = 50.

In [304]:
for image_name in all_image_names:

    img_normalized = load_data(
              DataManager.get_intensity_normalization_result_filepath(what='normalized_float_map', stack=stack, fn=image_name),
             download_s3=False)    
    
    t = time.time()
    img_normalized_uint8 = rescale_intensity_v2(img_normalized, low, high)
    sys.stderr.write('Rescale to uint8: %.2f seconds.\n' % (time.time() - t))

    t = time.time()
    raw_mask = load_data(DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, fn=image_name, version='mask', resol=resol, ext='bp'),
                        download_s3=False)
    img_normalized_uint8[~raw_mask] = 0
    sys.stderr.write('Load mask: %.2f seconds.\n' % (time.time() - t))
        
    img = 255 - img_normalized_uint8
    save_data(gamma_map[img], 
              DataManager.get_image_filepath_v2(stack=stack, prep_id=prep_id, fn=image_name, version=out_version, resol=resol),
             upload_s3=False)

Rescale to uint8: 35.48 seconds.
Load mask: 1.72 seconds.
Rescale to uint8: 41.11 seconds.
Load mask: 2.36 seconds.
Rescale to uint8: 27.04 seconds.
Load mask: 1.65 seconds.
Rescale to uint8: 29.19 seconds.
Load mask: 1.71 seconds.


In [ ]:
# (HUMAN) Give alignedWithMargin cropbox, based on alignedPadded images.
# or automatically infer based on alignedPadded masks.

In [260]:
image_name_list = ['MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301',
        'MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302',
        'MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304',
        'MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305'
                  ]

In [261]:
resol = 'down32'

bbox_all_images = []
for img_name in image_name_list:
    mask_tb_alignedPadded = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id='alignedPadded', fn=img_name)
    bbox = bbox_2d(mask_tb_alignedPadded)
    bbox_all_images.append(bbox)
bbox_all_images = np.array(bbox_all_images)

#     bbox_all_images = np.array([
#         bbox_2d(DataManager.load_thumbnail_mask_v3(stack=stack, prep_id=1, fn=fn))
#         for fn in metadata_cache['valid_filenames'][stack]
#     ])

# Are the bounding boxes reasonable? If some numbers stand out, go back to check the mask.
# plt.figure(figsize=(10,5));
# plt.plot(bbox_all_images[:,0], label='xmin')
# plt.plot(bbox_all_images[:,1], label='xmax')
# plt.plot(bbox_all_images[:,2], label='ymin')
# plt.plot(bbox_all_images[:,3], label='ymax')
# plt.legend();
# plt.show();

margin_um = 736
margin = int(np.round(margin_um / convert_resolution_string_to_um(resolution=resol, stack=stack)))
alignedWithMargin_xmin, alignedWithMargin_ymin = np.maximum(bbox_all_images[:, [0,2]].min(axis=0) - margin, 0)
alignedWithMargin_xmax, alignedWithMargin_ymax = np.minimum(bbox_all_images[:, [1,3]].max(axis=0) + margin, 
                                                            [mask_tb_alignedPadded.shape[1]-1, mask_tb_alignedPadded.shape[0]-1])

# print alignedWithMargin_xmin, alignedWithMargin_xmax, alignedWithMargin_ymin, alignedWithMargin_ymax 

alignedWithMargin_cropbox = {'rostral_limit': alignedWithMargin_xmin, 'caudal_limit': alignedWithMargin_xmax, 
'dorsal_limit': alignedWithMargin_ymin, 'ventral_limit': alignedWithMargin_ymax,
                            'resolution': resol, 'wrt': 'alignedPadded'}

print alignedWithMargin_cropbox

save_data(alignedWithMargin_cropbox, 
          DataManager.get_cropbox_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedWithMargin'))

{'ventral_limit': 645, 'caudal_limit': 1223, 'resolution': 'down32', 'wrt': 'alignedPadded', 'dorsal_limit': 55, 'rostral_limit': 10}


In [171]:
# for img_name in image_name_list:
#     mask_tb_alignedPadded = DataManager.load_thumbnail_mask_v3(stack=stack, prep_id='alignedPadded', fn=img_name)
#     plt.figure();
#     plt.imshow(mask_tb_alignedPadded[alignedWithMargin_ymin:alignedWithMargin_ymax+1, 
#                                      alignedWithMargin_xmin:alignedWithMargin_xmax+1], cmap=plt.cm.gray)
#     plt.show();

In [ ]:
# thumbnail -> prep1_thumbnail + prep1_thumbnail -> prep5_thumbnail

In [246]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = NtbNormalizedAdaptiveInvertedGamma
resol = thumbnail

Overwriting input_spec.ini


In [254]:
! python warp_crop.py --input_spec input_spec.ini \
 --warp "{toanchor_transforms_fp}" \
 --crop "{DataManager.get_cropbox_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedWithMargin')}" \
 --out_prep_id alignedWithMargin

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# raw -> prep1_raw + prep1_raw -> prep5_raw

In [255]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = None
version = NtbNormalizedAdaptiveInvertedGamma
resol = raw

Overwriting input_spec.ini


In [256]:
! python warp_crop.py --input_spec input_spec.ini \
 --warp "{toanchor_transforms_fp}" \
 --crop "{DataManager.get_cropbox_filename_v2(stack=stack, anchor_fn=None, prep_id='alignedWithMargin')}" \
 --out_prep_id alignedWithMargin

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep5_thumbnail_NtbNormalizedAdaptiveInvertedGamma

In [257]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = alignedWithMargin
version = NtbNormalizedAdaptiveInvertedGamma
resol = raw

Overwriting input_spec.ini


In [258]:
! python rescale.py input_spec.ini thumbnail -f {1./32}

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
/usr/local/lib/python2.7/dist-packages/skimage/external/tifffile/tifffile.py:2611: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
/data/CSHL_data_processed/DEMO998/DEMO998_prep5_raw_NtbNormalizedAdaptiveInvertedGamma/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_prep5_raw_NtbNormalizedAdaptiveInvertedGamma.tif
uint8
Rescale: 6.54 second

In [ ]:
# **(HUMAN)** Specify prep2 (alignedBrainstemCrop) cropping box, based on alignedWithMargin thumbnails or alignedPadded thumbnails

In [276]:
%%writefile {DataManager.get_cropbox_filename_v2(stack=stack, prep_id='alignedBrainstemCrop')}
{"rostral_limit": 419, "caudal_limit": 1148, "dorsal_limit": 128, "ventral_limit": 608, "resolution": "down32", "wrt": "alignedPadded"}

Overwriting /data/CSHL_data_processed/DEMO998/DEMO998_alignedTo_MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304_prep2_cropbox.json


In [ ]:
# prep5_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGamma

In [324]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = alignedWithMargin
version = NtbNormalizedAdaptiveInvertedGamma
resol = raw

Overwriting input_spec.ini


In [325]:
def convert_cropbox_from_arr_xywh_1um(data, out_fmt, out_resol, stack=None):

    data = data / convert_resolution_string_to_um(stack=stack, resolution=out_resol)

    if out_fmt == 'str_xywh':
        return ','.join(map(str, data))
    elif out_fmt == 'dict':
        raise Exception("too lazy to implement")
    else:
        return data

def convert_cropbox_to_arr_xywh_1um(data, in_fmt, in_resol, stack=None):

    if isinstance(data, dict):
        arr_xywh = np.array([data['rostral_limit'], data['dorsal_limit'], data['caudal_limit'] - data['rostral_limit'] + 1, data['ventral_limit'] - data['dorsal_limit'] + 1])
        # Since this does not check for wrt, the user needs to make sure the cropbox is relative to the input prep (i.e. the wrt attribute is the same as input prep)
    elif isinstance(data, str):
        if in_fmt == 'str_xywh':
            arr_xywh = np.array(map(int, map(eval, data.split(','))))
        elif in_fmt == 'str_xxyy':
            arr_xxyy = np.array(map(int, map(eval, data.split(','))))
            arr_xywh = np.array([arr_xxyy[0], arr_xxyy[2], arr_xxyy[1] - arr_xxyy[0] + 1, arr_xxyy[3] - arr_xxyy[2] + 1])
        else:
            raise
    else:
        if in_fmt == 'arr_xywh':
            arr_xywh = np.array(data)
        elif in_fmt == 'arr_xxyy':
            arr_xywh = np.array([data[0], data[2], data[1] - data[0] + 1, data[3] - data[2] + 1])
        else:
            raise
            
    arr_xywh_1um = arr_xywh * convert_resolution_string_to_um(stack=stack, resolution=in_resol)
    return arr_xywh_1um

def convert_cropbox_fmt(in_fmt, out_fmt, data, in_resol='1um', out_resol='1um', stack=None):
    arr_xywh_1um = convert_cropbox_to_arr_xywh_1um(data=data, in_fmt=in_fmt, in_resol=in_resol, stack=stack)
    return convert_cropbox_from_arr_xywh_1um(data=arr_xywh_1um, out_fmt=out_fmt, out_resol=out_resol, stack=stack)

In [327]:
! python warp_crop.py --input_spec input_spec.ini \
 --warp "{toanchor_transforms_fp}" \
 --crop "{convert_cropbox_fmt(data=DataManager.load_cropbox_v2_relative(stack=stack, prep_id='alignedBrainstemCrop', \
                                     wrt_prep_id='alignedWithMargin', \
                                    out_resolution='thumbnail'), \
                    in_fmt='arr_xxyy', out_fmt='str_xywh', stack=stack)}" \
 --out_prep_id alignedBrainstemCrop

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
rm -f /tmp/stderr_*; rm -f /tmp/stdout_*
return code: 0
Run locally.
Jobs submitted. Use wait_qsub_complete() to wait for all execution to finish.


In [ ]:
# prep2_raw_NtbNormalizedAdaptiveInvertedGamma -> prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg

In [328]:
! python compress_jpeg.py -h

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
usage: compress_jpeg.py [-h] [--depth DEPTH] [--quality QUALITY] input_spec

Compress image as JPEG. The output version is the input version with "Jpeg" appended.

positional arguments:
  input_spec         Input specifier

optional arguments:
  -h, --help         show this help message and exit
  --depth DEPTH      Image depth
  --quality QUALITY  JPEG quality


In [329]:
%%writefile input_spec.ini
[DEFAULT]
image_name_list = MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301
        MD662&661-F101-2017.06.06-22.05.45_MD661_2_0302
        MD662&661-F102-2017.06.06-22.30.50_MD661_1_0304
        MD662&661-F102-2017.06.06-22.30.50_MD661_2_0305
stack = DEMO998
prep_id = alignedBrainstemCrop
version = NtbNormalizedAdaptiveInvertedGamma
resol = raw

Overwriting input_spec.ini


In [330]:
! python compress_jpeg.py input_spec.ini

ENABLE_UPLOAD_S3 is not set, default to False.
ENABLE_DOWNLOAD_S3 is not set, default to False.
Setting environment for Precision WorkStation
No vtk
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_sorted_filenames.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
File does not exist: /data/CSHL_data_processed/DEMO999/DEMO999_anchor.txt
convert "/data/CSHL_data_processed/DEMO998/DEMO998_prep2_raw_NtbNormalizedAdaptiveInvertedGamma/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_prep2_raw_NtbNormalizedAdaptiveInvertedGamma.tif" -depth 8 -format jpg -quality 80 "/data/CSHL_data_processed/DEMO998/DEMO998_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/MD662&661-F101-2017.06.06-22.05.45_MD661_1_0301_prep2_raw_NtbNormali